In [1]:
import os
import sys

import numpy as np
import pandas as pd
import dotenv
import mlflow
from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame
import plotly.graph_objects as go
from huggingface_hub import login

sys.path.append("../..")

from utils import calculate_sklearn_metrics, TrainingConfig

dotenv.load_dotenv("../../.env")

token = os.environ["HF_TOKEN"]
login(token=token)

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("rosstat_forecasting");

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [2]:
data_dir = '../../../data/rosstat/processed'

train_df = pd.read_csv(os.path.join(data_dir, 'train/data.csv'))
val_df = pd.read_csv(os.path.join(data_dir, 'val/data.csv'))
test_df = pd.read_csv(os.path.join(data_dir, 'test/data.csv'))

print(f"Обучающая выборка: {train_df.shape[0]} строк")
print(f"Валидационная выборка: {val_df.shape[0]} строк")
print(f"Тестовая выборка: {test_df.shape[0]} строк")

Обучающая выборка: 4140 строк
Валидационная выборка: 828 строк
Тестовая выборка: 828 строк


In [3]:
train_data = TimeSeriesDataFrame.from_data_frame(
    train_df.rename(columns={"nominal_wage": "target"}),
    id_column="code",
    timestamp_column="date",
)
val_data = TimeSeriesDataFrame.from_data_frame(
    val_df.rename(columns={"nominal_wage": "target"}),
    id_column="code",
    timestamp_column="date",
)
test_data = TimeSeriesDataFrame.from_data_frame(
    test_df.rename(columns={"nominal_wage": "target"}),
    id_column="code",
    timestamp_column="date",
)

In [4]:
config = TrainingConfig(
    prediction_length=2,  # полгода
    artifact_path="../models/auto_ml_single_target",
)

predictor = TimeSeriesPredictor(
    prediction_length=config.prediction_length, path=config.artifact_path, freq="MS"
).fit(
    train_data=train_data,
    tuning_data=val_data,
    verbosity=4,
    hyperparameters={
        "DirectTabular": {},
        "RecursiveTabular": {},
        "TemporalFusionTransformer": {},
        "Chronos": [
            {"model_path": "bolt_small", "ag_args": {"name_suffix": "ZeroShot"}},
            {
                "model_path": "bolt_small",
                "fine_tune": True,
                "ag_args": {"name_suffix": "FineTuned"},
            },
        ],
    },
    enable_ensemble=False,
)

Beginning AutoGluon training...
AutoGluon will save models to '/home/nikita/projects/time_series_analysis/code_dir/rosstat/models/auto_ml_single_target'
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.12.7
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #60~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Fri Mar 28 16:09:21 UTC 2
CPU Count:          12
GPU Count:          1
Memory Avail:       22.34 GB / 30.95 GB (72.2%)
Disk Space Avail:   158.26 GB / 233.67 GB (67.7%)

Fitting with arguments:
{'enable_ensemble': False,
 'eval_metric': WQL,
 'freq': 'MS',
 'hyperparameters': {'Chronos': [{'ag_args': {'name_suffix': 'ZeroShot'},
                                  'model_path': 'bolt_small'},
                                 {'ag_args': {'name_suffix': 'FineTuned'},
                                  'fine_tune': True,
                                  'model_path': 'bolt_small'}],
                     'DirectTabular': {},
    

[1000]	valid_set's l1: 0.776516


No path specified. Models will be saved in: "AutogluonModels/ag-20250501_203841SeasonalNaive"
Shortening all time series to at most 2500
	-0.1321       = Validation score (-WQL)
	0.75    s     = Training runtime
	0.85    s     = Validation (prediction) runtime
Training timeseries model DirectTabular. 
Shortening all series to at most 14495
train_df shape: (4002, 20), val_df shape: (138, 20)
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.12.7
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #60~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Fri Mar 28 16:09:21 UTC 2
CPU Count:          12
Memory Avail:       21.33 GB / 30.95 GB (68.9%)
Disk Space Avail:   158.26 GB / 233.67 GB (67.7%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/st

In [5]:
config = TrainingConfig(
    prediction_length=2,  # полгода
    artifact_path="../models/auto_ml_single_target",
)

predictor = TimeSeriesPredictor.load(config.artifact_path)

Loading predictor from path /home/nikita/projects/time_series_analysis/code_dir/rosstat/models/auto_ml_single_target


In [6]:
leaderboard = predictor.leaderboard(
    test_data,
    extra_metrics=['MASE', 'MAPE', 'MSE', 'MAE', 'SQL'],
)
leaderboard.rename(columns={'score_test': 'WQL_test', 'score_val': 'WQL_val'}, inplace=True)
leaderboard

Generating leaderboard for all models trained
Additional data provided, testing on additional data. Resulting leaderboard will be sorted according to test score (`score_test`).


Found no cached predictions
Prediction order: ['ChronosZeroShot[bolt_small]', 'ChronosFineTuned[bolt_small]', 'DirectTabular', 'RecursiveTabular', 'TemporalFusionTransformer']
loading configuration file config.json from cache at /home/nikita/.cache/huggingface/hub/models--autogluon--chronos-bolt-small/snapshots/94d26f8f14f17f8c7c6ddf01521a959d4722bc6e/config.json
Model config T5Config {
  "architectures": [
    "ChronosBoltModelForForecasting"
  ],
  "chronos_config": {
    "context_length": 2048,
    "input_patch_size": 16,
    "input_patch_stride": 16,
    "prediction_length": 64,
    "quantiles": [
      0.1,
      0.2,
      0.3,
      0.4,
      0.5,
      0.6,
      0.7,
      0.8,
      0.9
    ],
    "use_reg_token": true
  },
  "chronos_pipeline_class": "ChronosBoltPipeline",
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "re

,model,WQL_test,WQL_val,pred_time_test,pred_time_val,fit_time_marginal,fit_order,MASE,MAPE,MSE,MAE,SQL
0,TemporalFusionTransformer,-0.089873,-0.067373,0.045958,0.440540,20.096687,5,-9871.897486,-0.081069,-3.058447e+08,-9871.897486,-8428.169641
1,ChronosFineTuned[bolt_small],-0.119558,-0.122942,0.074033,0.008323,29.522093,4,-13770.490591,-0.120746,-4.214574e+08,-13770.490591,-11211.966630
2,RecursiveTabular,-0.135458,-0.132127,1.048662,0.851270,0.748199,1,-14335.877536,-0.121996,-5.510396e+08,-14335.877536,-12703.014848
3,ChronosZeroShot[bolt_small],-0.135828,-0.134195,0.660885,0.865939,0.884435,3,-14345.202859,-0.121813,-5.634949e+08,-14345.202859,-12737.699989
4,DirectTabular,-0.157112,-0.141253,0.231305,0.230250,13.578918,2,-17287.149689,-0.161020,-6.080210e+08,-17287.149689,-14733.676028


In [7]:
predictions = predictor.predict(val_data, 
                                model='ChronosFineTuned[bolt_small]',)
                                # known_covariates=prediction_covariates)

Found no cached predictions
Prediction order: {'ChronosFineTuned[bolt_small]'}


	Fine-tuned checkpoint exists, setting model_path to /home/nikita/projects/time_series_analysis/code_dir/rosstat/models/auto_ml_single_target/models/ChronosFineTuned[bolt_small]/fine-tuned-ckpt
loading configuration file /home/nikita/projects/time_series_analysis/code_dir/rosstat/models/auto_ml_single_target/models/ChronosFineTuned[bolt_small]/fine-tuned-ckpt/config.json
Model config T5Config {
  "architectures": [
    "ChronosBoltModelForForecasting"
  ],
  "chronos_config": {
    "context_length": 2048,
    "input_patch_size": 16,
    "input_patch_stride": 16,
    "prediction_length": 64,
    "quantiles": [
      0.1,
      0.2,
      0.3,
      0.4,
      0.5,
      0.6,
      0.7,
      0.8,
      0.9
    ],
    "use_reg_token": true
  },
  "chronos_pipeline_class": "ChronosBoltPipeline",
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_pr

In [8]:
def extract_specific_rows_from_indexed_data(data, start_row: int, end_row: int):
    rows_to_extract = np.arange(start_row, end_row)
    unique_ids = data.index.get_level_values('item_id').unique()
        
    selected_data = []

    for item_id in unique_ids:
        item_data = data.loc[[item_id]]
        
        selected_rows = item_data.iloc[rows_to_extract]
        selected_data.append(selected_rows)

    result = pd.concat(selected_data)

    return result

k = 12

top_k_models = leaderboard.sort_values(['SQL'], ascending=False).head(k)['model'].tolist()
window_size = config.prediction_length
test_length = test_df['code'].value_counts().iloc[0]
max_iterations = (test_length + window_size - 1) // window_size# - 1 ещё -1 из-за known_covariates

current_data = train_data.copy()
all_val_models_predictions = {}

for i in range(max_iterations):
    start_idx = i * window_size
    end_idx = start_idx + window_size
    
    for model_name in top_k_models:
        if model_name not in all_val_models_predictions:
            all_val_models_predictions[model_name] = []
        
        # future_covariates = test_data[start_idx:start_idx + config.prediction_length][known_covariates_names]
        # prediction_covariates = pd.concat([current_data[known_covariates_names], future_covariates])
        
        predictions = predictor.predict(current_data, 
                                       model=model_name,)
                                       # known_covariates=prediction_covariates)
                                       
        all_val_models_predictions[model_name].append(predictions)
        
    current_data = pd.concat([current_data, extract_specific_rows_from_indexed_data(val_data, start_idx, end_idx)])

test_df_shape = test_df.shape[0]
all_val_models_predictions = {k: pd.concat(v)[:test_df_shape] for k, v in all_val_models_predictions.items()}

current_data = val_data.copy()
all_test_models_predictions = {}

for i in range(max_iterations):
    start_idx = i * window_size
    end_idx = start_idx + window_size
    
    for model_name in top_k_models:
        if model_name not in all_test_models_predictions:
            all_test_models_predictions[model_name] = []
        
        # future_covariates = test_data[start_idx:start_idx + config.prediction_length][known_covariates_names]
        # prediction_covariates = pd.concat([current_data[known_covariates_names], future_covariates])
        
        predictions = predictor.predict(current_data, 
                                       model=model_name,)
                                       # known_covariates=prediction_covariates)
                                       
        all_test_models_predictions[model_name].append(predictions)
        
    current_data = pd.concat([current_data, extract_specific_rows_from_indexed_data(test_data, start_idx, end_idx)])

test_df_shape = test_df.shape[0]
all_test_models_predictions = {k: pd.concat(v)[:test_df_shape] for k, v in all_test_models_predictions.items()}

Found no cached predictions
Prediction order: {'TemporalFusionTransformer'}


Extending existing cached predictions
Cached predictions saved to /home/nikita/projects/time_series_analysis/code_dir/rosstat/models/auto_ml_single_target/models/cached_predictions.pkl
Loaded cached predictions for models ['TemporalFusionTransformer']
Prediction order: {'ChronosFineTuned[bolt_small]'}
	Fine-tuned checkpoint exists, setting model_path to /home/nikita/projects/time_series_analysis/code_dir/rosstat/models/auto_ml_single_target/models/ChronosFineTuned[bolt_small]/fine-tuned-ckpt
loading configuration file /home/nikita/projects/time_series_analysis/code_dir/rosstat/models/auto_ml_single_target/models/ChronosFineTuned[bolt_small]/fine-tuned-ckpt/config.json
Model config T5Config {
  "architectures": [
    "ChronosBoltModelForForecasting"
  ],
  "chronos_config": {
    "context_length": 2048,
    "input_patch_size": 16,
    "input_patch_stride": 16,
    "prediction_length": 64,
    "quantiles": [
      0.1,
      0.2,
      0.3,
      0.4,
      0.5,
      0.6,
      0.7,
   

In [9]:
from plotly.subplots import make_subplots

def plot_forecasts_val_test(
    val_df: pd.DataFrame,
    test_df: pd.DataFrame,
    val_predictions: pd.DataFrame,
    test_predictions: dict[str, pd.DataFrame],
    start_date: str | None = None,
    end_date: str | None = None,
    height: int = 300,
    width: int = 1400,
    item_id: str | None = None,
):
    model_names = list(test_predictions.keys())
    n_models = len(model_names)
    rows = int(np.ceil(n_models / 2))
    cols = 2
    
    filtered_val_df = val_df.copy()
    filtered_test_df = test_df.copy()
    
    if start_date is not None and end_date is not None:        
        test_mask = (filtered_test_df["timestamp"] >= start_date) & (filtered_test_df["timestamp"] <= end_date)
        filtered_test_df = filtered_test_df[test_mask]
    
    fig = make_subplots(
        rows=rows, cols=cols, subplot_titles=model_names, vertical_spacing=0.1
    )
    
    for i, model_name in enumerate(model_names):
        row = i // 2 + 1
        col = i % 2 + 1

        model_val_pred = val_predictions[model_name].copy()
        if item_id is not None:
            model_val_pred = model_val_pred.loc[item_id]

        filtered_val_mean = model_val_pred["mean"].values
        filtered_val_upper = model_val_pred["0.9"].values if "0.9" in model_val_pred else None
        filtered_val_lower = model_val_pred["0.1"].values if "0.1" in model_val_pred else None
        
        model_test_pred = test_predictions[model_name]
        if item_id is not None:
            model_test_pred = model_test_pred.loc[item_id]
        
        if start_date is not None and end_date is not None:
            test_time_mask = (test_df["timestamp"] >= start_date) & (test_df["timestamp"] <= end_date)
            
            filtered_test_mean = model_test_pred["mean"].values[test_time_mask]
            filtered_test_upper = model_test_pred["0.9"].values[test_time_mask] if "0.9" in model_test_pred else None
            filtered_test_lower = model_test_pred["0.1"].values[test_time_mask] if "0.1" in model_test_pred else None
            
        else:
            filtered_test_mean = model_test_pred["mean"]
            filtered_test_upper = model_test_pred["0.9"] if "0.9" in model_test_pred else None
            filtered_test_lower = model_test_pred["0.1"] if "0.1" in model_test_pred else None
        
        fig.add_trace(
            go.Scatter(
                x=filtered_val_df["timestamp"],
                y=filtered_val_df["target"],
                name="Validation (actual)",
                line=dict(color="blue"),
            ),
            row=row,
            col=col,
        )
        
        fig.add_trace(
            go.Scatter(
                x=filtered_val_df["timestamp"],
                y=filtered_val_mean,
                name="Validation (predicted)",
                line=dict(color="purple", dash="dot"),
            ),
            row=row,
            col=col,
        )
        
        fig.add_trace(
            go.Scatter(
                x=filtered_test_df["timestamp"],
                y=filtered_test_df["target"],
                name="Test (actual)",
                line=dict(color="#50C878"),
            ),
            row=row,
            col=col,
        )
        
        fig.add_trace(
            go.Scatter(
                x=filtered_test_df["timestamp"],
                y=filtered_test_mean,
                name=f"{model_name} Test (predicted)",
                line=dict(color="#D70040", dash="dot"),
            ),
            row=row,
            col=col,
        )

        fig.add_trace(
                go.Scatter(
                    x=filtered_val_df["timestamp"],
                    y=filtered_val_upper,
                    mode="lines",
                    line=dict(width=0),
                    showlegend=False,
                ),
                row=row,
                col=col,
            )
        fig.add_trace(
            go.Scatter(
                x=filtered_val_df["timestamp"],
                y=filtered_val_lower,
                mode="lines",
                fill="tonexty",
                fillcolor="rgba(128, 0, 128, 0.6)",
                line=dict(width=0),
                name=f"{model_name} CI (0.1-0.9)",
            ),
            row=row,
            col=col,
        )
        
        if filtered_test_upper is not None and filtered_test_lower is not None:
            fig.add_trace(
                go.Scatter(
                    x=filtered_test_df["timestamp"],
                    y=filtered_test_upper,
                    mode="lines",
                    line=dict(width=0),
                    showlegend=False,
                ),
                row=row,
                col=col,
            )
            fig.add_trace(
                go.Scatter(
                    x=filtered_test_df["timestamp"],
                    y=filtered_test_lower,
                    mode="lines",
                    fill="tonexty",
                    fillcolor="rgba(255, 127, 14, 0.6)",
                    line=dict(width=0),
                    name=f"{model_name} CI (0.1-0.9)",
                ),
                row=row,
                col=col,
            )
    
    fig.update_layout(
        title="Forecasts with Validation and Test Data",
        template="plotly_white",
        height=height * rows,
        width=width,
        showlegend=True,
    )
    
    for i in range(1, rows * cols + 1):
        fig.update_xaxes(
            title_text="Date",
            row=i // cols + 1,
            col=i % cols if i % cols != 0 else cols,
        )
        fig.update_yaxes(
            title_text="National Demand",
            row=i // cols + 1,
            col=i % cols if i % cols != 0 else cols,
        )
    
    fig.show()

In [10]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import datetime
from utils.plotting import plot_forecasts

date_col = pd.to_datetime(test_df["date"])
min_date = date_col.min().date()
max_date = date_col.max().date()
height = 400
width = 1200

start_date_picker = widgets.DatePicker(
    description="Start date:", disabled=False, value=min_date
)

end_date_picker = widgets.DatePicker(
    description="End date:", disabled=False, value=max_date
)

output_area = widgets.Output()


def on_button_clicked(b):
    with output_area:
        clear_output(wait=True)
        start_date = datetime.datetime.combine(
            start_date_picker.value, datetime.datetime.min.time()
        )
        end_date = datetime.datetime.combine(
            end_date_picker.value, datetime.datetime.min.time()
        )
        plot_forecasts_val_test(
            val_df=val_df_,
            test_df=test_df_,
            val_predictions=all_val_models_predictions_,
            test_predictions=all_test_models_predictions,
            start_date=start_date,
            end_date=end_date,
            height=height,
            width=width,
            item_id=item_id,
        )


plot_button = widgets.Button(description="Plot Forecasts")
plot_button.on_click(on_button_clicked)

controls = widgets.VBox(
    [widgets.HBox([start_date_picker, end_date_picker]), plot_button]
)

display(controls, output_area)

# item_id='82.91'
# item_id='01.11.39'
item_id='82.99'
item_id = 81

val_df_ = val_df.rename(columns={'date': 'timestamp', "nominal_wage": "target"})[['code', 'timestamp', "target"]]
val_df_ = val_df_[val_df_['code'].eq(item_id)].reset_index(drop=True)
val_df_['timestamp'] = pd.to_datetime(val_df_['timestamp'])

test_df_ = test_df.rename(columns={'date': 'timestamp', "nominal_wage": "target"})[['code', 'timestamp', "target"]]
test_df_ = test_df_[test_df_['code'].eq(item_id)].reset_index(drop=True)
test_df_['timestamp'] = pd.to_datetime(test_df_['timestamp'])

val_df_ = pd.concat([val_df_, test_df_.iloc[[0]]])

all_val_models_predictions_ = all_val_models_predictions.copy()
for model in all_val_models_predictions_.keys():
    all_val_models_predictions_[model] = pd.concat([all_val_models_predictions_[model], all_test_models_predictions[model].loc[[item_id]].iloc[[0]]])

with output_area:
    plot_forecasts_val_test(
        val_df=val_df_,
        test_df=test_df_,
        val_predictions=all_val_models_predictions_,
        test_predictions=all_test_models_predictions,
        height=height,
        width=width,
        item_id=item_id,
    )

Output()

In [11]:
all_codes = test_df['code'].unique()

In [25]:
test_df[test_df["code"].eq(code)][["nominal_wage"]]

,nominal_wage
12,45308.4
13,48720.4
14,49383.2
15,49534.2
16,48433.7
17,50774.0
18,53697.0
19,51631.7
20,54851.8
21,55021.4


In [33]:
all_test_models_predictions.keys()

dict_keys(['TemporalFusionTransformer', 'ChronosFineTuned[bolt_small]', 'RecursiveTabular', 'ChronosZeroShot[bolt_small]', 'DirectTabular'])

In [38]:
all_models_metrics = {}

for model in all_test_models_predictions.keys():
    metrics_df = []
    for code in all_codes:
        pred_df = pd.concat([
            all_test_models_predictions[model]
            .loc[code][["0.1", "0.5", "0.9"]]
            .reset_index(drop=True),
            test_df[test_df["code"].eq(code)][["nominal_wage"]].reset_index(drop=True),
        ], axis=1)
        pred_df = pd.DataFrame(pred_df)

        metrics_df.append(calculate_sklearn_metrics(pred_df, target_column='nominal_wage'))

    metrics_dict = pd.DataFrame(metrics_df).mean().to_dict()

    all_models_metrics[model] = metrics_dict

In [40]:
prefix = 'AutoGluon'

for k, metrics_ in all_models_metrics.items():
    run_name = f"{k}_{prefix}"

    with mlflow.start_run(run_name=run_name):
        mlflow.log_metrics(metrics_)
        mlflow.log_param("model_name", model_name)

        mlflow.set_tag("prefix", prefix)

🏃 View run TemporalFusionTransformer_AutoGluon at: http://127.0.0.1:5000/#/experiments/169882278836627198/runs/8c618c134b454ba68f2aa7d8b015048e
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/169882278836627198
🏃 View run ChronosFineTuned[bolt_small]_AutoGluon at: http://127.0.0.1:5000/#/experiments/169882278836627198/runs/a43e818d33274e17aecae589c54e5562
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/169882278836627198
🏃 View run RecursiveTabular_AutoGluon at: http://127.0.0.1:5000/#/experiments/169882278836627198/runs/f8fedfd1a3374defa0b511dcac72c4d9
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/169882278836627198
🏃 View run ChronosZeroShot[bolt_small]_AutoGluon at: http://127.0.0.1:5000/#/experiments/169882278836627198/runs/970f5e30ef45474fb87a664037410031
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/169882278836627198
🏃 View run DirectTabular_AutoGluon at: http://127.0.0.1:5000/#/experiments/169882278836627198/runs/18a041662fd149d3a896e